**How this will work** <br>


1.   A user will provide a hypothesis and a document and the model will then infer if there is any clause in the document that support, contradict or not mentioned(neutral) the hypothesis
2.   Model will be batch trained. Each batch will represent one document and one hypothesis. 
> Note that in a document there can be 100's of clauses but just one or two clause may support or contradict the hypothesis. This implies over 90% of clauses will be neutral to the hypothesis. This makes data highly imbalanced <br>
> I have a parameter max_neutral that controles how many neutral cases we want to add







In [113]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
import os
dir= "/content/drive/MyDrive/Github/CS-7643/project/contract-nli"
os.chdir(dir)

In [3]:
#!pip install bert-pytorch
!pip install sentencepiece
!pip install transformers
!pip install onnx
!pip install fasttext
!pip install torchtext
!pip install scikit-learn
# https://huggingface.co/docs/transformers/preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.4 MB/s 
     |████████████████████████████████| 163 kB 63.3 MB/s 
     |████████████████████████████████| 7.6 MB 49.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.2 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3157960 sha256=c2701a72c378e2b2bd7d88f017f79620d1508deb77213f464a48463e725a3d80
  Stored in directory: /root/.cache/

In [4]:
import json
import pickle
import pandas as pd
import re 
import transformers
import torch
import numpy as np
from torchtext.data import get_tokenizer
from transformers import BertTokenizer, BertModel, BertConfig
from torch.utils.data import Dataset, DataLoader

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn.metrics import pairwise

In [115]:
# This will take time
d=100
from torchtext.vocab import GloVe
#global_vectors = GloVe(name='840B', dim=d)
global_vectors = GloVe(name='6B', dim=d)

In [116]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
nltk.download('omw-1.4')
import re
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

mlngt = 100 #max_length,
mn =10 #max_neutral

def clean_head(ab):
    ab = ab.replace(","," ,").replace(";"," ;").replace(":"," :")
    ab = ab.lower().strip()    
    #marked_text = "[CLS] " + ab + " [SEP]"
    # Tokenize our sentence with the BERT tokenizer.
    #ab = tokenizer.tokenize(marked_text) 
    return ab

def preprocess(sentence):
  sentence=str(sentence)
  sentence = sentence.lower().replace(","," ,").replace(";"," ;").replace(":"," :").replace("__"," ").replace("  "," ")
  sentence=sentence.replace('{html}',"") 
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', sentence)
  rem_url=re.sub(r'http\S+', '',cleantext)
  rem_num = re.sub('[0-9]+', '', rem_url)
  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(rem_num)  
  #filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
  #stem_words=[stemmer.stem(w) for w in filtered_words]
  #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
  return " ".join(tokens)



def getPositionEncoding(max_length, d, n=10000):
    P = np.zeros((max_length, d))
    #print(max_length,P.shape)
    for k in range(max_length):      
      for i in np.arange(int(d/2)):
        denominator = np.power(n, 2*i/d)
        P[k, 2*i] = np.sin(k/denominator)
        P[k, 2*i+1] = np.cos(k/denominator)
    return P
Pg =  getPositionEncoding(mlngt, d)  
Pg  = torch.FloatTensor(Pg)

def emd(sent, max_length=mlngt):
  tok = get_tokenizer("basic_english")
  tkn = tok(sent)  
  ln = len(tkn)
  x = global_vectors.get_vecs_by_tokens(tkn)
  #print(x.shape, Pg.shape)
  if ln<max_length:  
    x = torch.nn.ConstantPad2d((0, 0, 0, max_length-ln), 0)(x)
  else:
    x= x[:max_length,:]
  x=x+Pg 
  return x

def emdb(sent):
  # Tokenize sentences
  encoded_input = tokenizer(sent, padding=True, truncation=True, return_tensors='pt')

  # Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Perform pooling. In this case, max pooling.
  x = mean_pooling(model_output, encoded_input['attention_mask'])
  seg = torch.tensor([0,1])
  seg = seg.tile((768,1)).t()    
  x=x+seg  
  return x

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [117]:
ab = emd("we are the world")
ab.shape

torch.Size([100, 100])

In [118]:
#################
# Opening JSON file
f = open('train.json')  
# returns JSON object as 
# a dictionary
data = json.load(f)

kes=['nda-11', 'nda-16', 'nda-15', 'nda-10', 'nda-2', 'nda-1', 'nda-19', 'nda-12', 'nda-20', 'nda-3', 'nda-18', 'nda-7', 'nda-17', 'nda-8', 'nda-13', 'nda-5', 'nda-4']
rg = len(data['documents'])
reference_list=[]
idx=0
for idx in range(rg):
  #Document level
  docid = data['documents'][idx]['id']
  for ndas_key in kes:
    reference_list.append([docid, ndas_key,idx])
  idx=idx+1
ref_train = pd.DataFrame(reference_list ,columns = ['docid', 'nda_key', 'idx'])
f.close()
##############
# Opening JSON file
f = open('test.json')  
# returns JSON object as 
# a dictionary
dataT = json.load(f)

kes=['nda-11', 'nda-16', 'nda-15', 'nda-10', 'nda-2', 'nda-1', 'nda-19', 'nda-12', 'nda-20', 'nda-3', 'nda-18', 'nda-7', 'nda-17', 'nda-8', 'nda-13', 'nda-5', 'nda-4']
rg = len(dataT['documents'])
reference_list=[]
idxt=0
for idxt in range(rg):
  #Document level
  docid = dataT['documents'][idxt]['id']

  for ndas_key in kes:
    reference_list.append([docid, ndas_key, idxt])
  idxt=idxt+1
ref_test = pd.DataFrame(reference_list ,columns = ['docid', 'nda_key', 'idx'])

In [119]:
# Do Not delete
def get_data(data, idx,ke, max_neutral=mn):
  dataM=[]
  #Document level
  docid = data['documents'][idx]['id']
  
  string = data['documents'][idx]['text']
  file_name = data['documents'][idx]['file_name']
  spans = data['documents'][idx]['spans']
  spanall=[]
  for span in spans:           
      spanval = string[span[0]: span[1]]
      spanall.append(spanval)
  
  ndas = data['documents'][idx]['annotation_sets'][0]['annotations'] 
  # Keys level
    
  hypothesis = data['labels'][ke]['hypothesis']  

  # Key level   

  choice =ndas[ke] ['choice']
  spansC = ndas[ke] ['spans']    
  for si in range(len(spanall)):
    span_nbr =si
    if si in spansC:
      val=choice
    else:
      val="Neutral"
    premise = spanall[si]
    itm = [docid, file_name , ke, hypothesis, span_nbr, premise,val ]
    dataM.append(itm)
  df = pd.DataFrame(dataM ,columns = ['docid', 'file_name', 'hypotheis_key', 'hypotheis', 'span_nbr', 'premise', 'choice'   ])
  #"[CLS] " and " [SEP] "" [SEP]" 
  df['premise'] = df['premise'].apply(preprocess)
  df['hypotheis']= df['hypotheis'].apply(preprocess) 
  df['label'] = df['choice'].map(lambda s: 0 if s=='Entailment' else (1 if s== 'Contradiction' else 2 ))
  df['entl'] =  df['choice'].map(lambda s : 1 if s== 'Entailment' else 0)
  df['cont'] =  df['choice'].map(lambda s : 1 if s== 'Contradiction' else 0)
  df['neut'] =  df['choice'].map(lambda s : 1 if s== 'Neutral' else 0)
  
  df1=df[df.choice !='Neutral']
  df2=df[df.choice =='Neutral']
  n = min(max_neutral, len(df2)-1)
  df2= df2.sample(n = n)
  df = pd.concat([df1, df2], sort=False)
  df = df[df.span_nbr !=-1 ][df.premise != ''] [df.hypotheis != '']
  df['bert_sent']= df.apply(lambda s: [s.premise,s.hypotheis], axis=1)
  return df

def create_completedf(data, refdf, max_neutral=50):
  cl = get_data(data, 0,'nda-1', 1)
  cols = list(cl.columns)
  df1= pd.DataFrame(columns=cols)
  for ind, row in refdf.iterrows():    
    df2 = get_data(data, row["idx"],row["nda_key"], max_neutral)
    df1=pd.concat([df1,df2])
  return df1



In [81]:
from sklearn.model_selection import train_test_split
ref_test, ref_valid = train_test_split(ref_test, test_size=0.5)


In [82]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [83]:
class MyDataset(Dataset): 
  def __init__(self,data, df, max_neutral=mn):       
    self.df =df.sample(frac=1)
    self.data =data
    self.max_neutral= max_neutral

  def __len__(self):
    return len(self.df)
  
  def __getitem__(self,idx):
    # Is should have used docid but I am using index in json. Check how ref files are structured
    docidx = self.df.iloc[idx]['idx']
    ndakey = self.df.iloc[idx]['nda_key']
    df1 = get_data(data, docidx, ndakey,self.max_neutral)
    label = torch.tensor(df1.label.values)
    premise = torch.stack(df1.premise.apply(emd).tolist())
    hypothesis =  torch.stack(df1.hypotheis.apply(emd).tolist())
    #bertsent  = torch.stack(df1.bert_sent.apply(emdb).tolist())
    
    #sample = {"Label": label.to(device), "Premise": premise.to(device) , "Hypothesis" : hypothesis.to(device), "Bert_Sent": bertsent.to(device)}
    sample = {"Label": label.to(device), "Premise": premise.to(device) , "Hypothesis" : hypothesis.to(device)}
    return sample

# Custome collate function
def coll(sample1):  
  if len(sample1)<2:
    return sample1
  else:
    label0 = sample1[0]['Label'].to(device)
    premise0 = sample1[0]['Premise'].to(device)
    hypothesis0 = sample1[0]['Hypothesis'].to(device)
    #bertsent0 = sample1[0]["Bert_Sent"].to(device)
    for i in range(1, len(sample1)):
      labeli = sample1[i]['Label'].to(device)
      premisei = sample1[i]['Premise'].to(device)
      hypothesisi = sample1[i]['Hypothesis'].to(device)
      #bertsenti = sample1[i]["Bert_Sent"].to(device)
      label0 =torch.cat((label0,labeli))
      premise0 = torch.cat((premise0, premisei))
      hypothesis0 = torch.cat((hypothesis0,hypothesisi))
      #bertsent0 = torch.cat((bertsent0, bertsenti))
    #sample1 = {"Label": label0, "Premise": premise0 , "Hypothesis" : hypothesis0, "Bert_Sent": bertsent0}
    sample1 = {"Label": label0, "Premise": premise0 , "Hypothesis" : hypothesis0}
    return sample1
    

In [84]:
train_data=MyDataset(data, ref_train)
train_loader=DataLoader(train_data,batch_size=20,collate_fn=coll, shuffle=False)
valid_data=MyDataset(dataT, ref_valid,max_neutral=2)
valid_loader=DataLoader(valid_data,batch_size=20,collate_fn=coll, shuffle=True)
test_data=MyDataset(dataT, ref_test,max_neutral=2)
test_loader=DataLoader(test_data,batch_size=20,collate_fn=coll, shuffle=True)


In [86]:
print(x['Label'].shape,x['Premise'].shape,x['Hypothesis'].shape)

torch.Size([217]) torch.Size([217, 100, 100]) torch.Size([217, 100, 100])


In [87]:
!pip install d2l==1.0.0-alpha1.post0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [89]:
def mlp(num_inputs, num_hiddens, flatten):
    net = []
    net.append(nn.Dropout(0.2))
    net.append(nn.Linear(num_inputs, num_hiddens))
    net.append(nn.ReLU())
    if flatten:
        net.append(nn.Flatten(start_dim=1))
    net.append(nn.Dropout(0.2))
    net.append(nn.Linear(num_hiddens, num_hiddens))
    net.append(nn.ReLU())
    if flatten:
        net.append(nn.Flatten(start_dim=1))
    return nn.Sequential(*net)

In [90]:
class Attend(nn.Module):
    def __init__(self, num_inputs, num_hiddens, **kwargs):
        super(Attend, self).__init__(**kwargs)
        self.f = mlp(num_inputs, num_hiddens, flatten=False)

    def forward(self, A, B):
        # Shape of `A`/`B`: (`batch_size`, no. of tokens in sequence A/B,
        # `embed_size`)
        # Shape of `f_A`/`f_B`: (`batch_size`, no. of tokens in sequence A/B,
        # `num_hiddens`)
        f_A = self.f(A)
        f_B = self.f(B)
        # Shape of `e`: (`batch_size`, no. of tokens in sequence A,
        # no. of tokens in sequence B)
        e = torch.bmm(f_A, f_B.permute(0, 2, 1))
        # Shape of `beta`: (`batch_size`, no. of tokens in sequence A,
        # `embed_size`), where sequence B is softly aligned with each token
        # (axis 1 of `beta`) in sequence A
        beta = torch.bmm(F.softmax(e, dim=-1), B)
        # Shape of `alpha`: (`batch_size`, no. of tokens in sequence B,
        # `embed_size`), where sequence A is softly aligned with each token
        # (axis 1 of `alpha`) in sequence B
        alpha = torch.bmm(F.softmax(e.permute(0, 2, 1), dim=-1), A)
        return beta, alpha

In [91]:
class Compare(nn.Module):
    def __init__(self, num_inputs, num_hiddens, **kwargs):
        super(Compare, self).__init__(**kwargs)
        self.g = mlp(num_inputs, num_hiddens, flatten=False)

    def forward(self, A, B, beta, alpha):
        V_A = self.g(torch.cat([A, beta], dim=2))
        V_B = self.g(torch.cat([B, alpha], dim=2))
        return V_A, V_B

In [92]:
class Aggregate(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs, **kwargs):
        super(Aggregate, self).__init__(**kwargs)
        self.h = mlp(num_inputs, num_hiddens, flatten=True)
        self.linear = nn.Linear(num_hiddens, num_outputs)

    def forward(self, V_A, V_B):
        # Sum up both sets of comparison vectors
        V_A = V_A.sum(dim=1)
        V_B = V_B.sum(dim=1)
        # Feed the concatenation of both summarization results into an MLP
        Y_hat = self.linear(self.h(torch.cat([V_A, V_B], dim=1)))
        return Y_hat

In [99]:
class DecomposableAttention(nn.Module):
    def __init__(self,  embed_size, num_hiddens, num_inputs_attend=100,
                 num_inputs_compare=200, num_inputs_agg=400, **kwargs):
        super(DecomposableAttention, self).__init__(**kwargs)
        #self.embedding = nn.Embedding(len(vocab), embed_size)
        self.attend = Attend(num_inputs_attend, num_hiddens)
        self.compare = Compare(num_inputs_compare, num_hiddens)
        # There are 3 possible outputs: entailment, contradiction, and neutral
        self.aggregate = Aggregate(num_inputs_agg, num_hiddens, num_outputs=3)

    def forward(self, X):
        A, B = X['Premise'], X['Hypothesis']
        beta, alpha = self.attend(A, B)
        V_A, V_B = self.compare(A, B, beta, alpha)
        Y_hat = self.aggregate(V_A, V_B)
        return Y_hat

In [ ]:
import torch.optim as optim
net = DecomposableAttention(100, 200)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    print(f"running epoch {epoch}")
    for i, x in enumerate(train_loader):

        # get the inputs; data is a list of [inputs, labels]
        print(f"running i {i}")
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net.forward(x)
        loss = criterion(outputs, x["Label"])
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
print('Finished Training')

running epoch 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


running i 0
running i 1
running i 2
running i 3
running i 4
running i 5
running i 6
running i 7
running i 8
running i 9
running i 10
running i 11
running i 12
running i 13
running i 14
running i 15
running i 16
running i 17
running i 18
running i 19
running i 20
running i 21
running i 22
running i 23
running i 24
running i 25
running i 26
running i 27
running i 28
running i 29
running i 30
running i 31
running i 32
running i 33
running i 34
running i 35
running i 36


**________________ END--------------------------**